## Import Packages:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
from astropy.io import ascii
import corner
import os
from timeit import default_timer as timer
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.lines as mlines
from scipy.optimize import least_squares, curve_fit
from scipy.stats import f
from astropy.table import Table, vstack
import pandas as pd
import lmfit
from matplotlib import rcParams
from matplotlib.colors import LinearSegmentedColormap

## Define relations to calculate B, R, and Mdot:

SSA dominant absorption:

In [21]:
# B field calculation for SSA dominated absorption
def B_peak_SSA(p, F_p, D, nu_p, F_p_scale, D_scale, nu_p_scale):
    power = (-4 / ((2 * p) + 13))
    B = ((p - 2)**power) * ((F_p/F_p_scale)**(power/2)) * ((D/D_scale)**power) * (nu_p/nu_p_scale)
    return B

# shock radius for SSA dominated absorption
def R_peak_SSA(p, F_p, D, nu_p, F_p_scale, D_scale, nu_p_scale):
    R = 8.8*10**15 * ((p - 2)**(-1 / ((2 * p) + 13))) * ((F_p/F_p_scale)**((p + 6)/((2 * p) + 13))) * ((D/D_scale)**(((2 * p) + 12)/((2 * p) + 13))) * ((nu_p/nu_p_scale)**(-1))
    return R

# mass loss rate at a given epoch for SSA dominated absorption
def Mdot_peak_SSA(m_H, B, t, v_wind, B_scale, t_scale, v_wind_scale):
    Mdot = ((6*10**(-7))/(m_H**2)) * ((B/B_scale)**2) * ((t/t_scale)**2) * ((v_wind/v_wind_scale))
    return Mdot

FFA dominant absorption:

In [5]:
# mass loss rate at a given epoch for FFA dominated absorption, assuming fully, singly ionized wind
def Mdot_peak_FFA(tau, v_ej, t, T_e, v_wind, v_ej_scale, t_scale, T_e_scale, v_wind_scale):
    Mdot = (4.76 * 10**(-5)) * (tau**(0.5)) * ((v_ej/v_ej_scale)**1.5) * ((t/t_scale)**1.5) * ((T_e/T_e_scale)**0.675) * (v_wind/v_wind_scale)
    return Mdot

Synchrotron cooling dominant:

In [37]:
def nu_syn(epsilon_B, Mdot, v_wind, epsilon_B_scale, Mdot_scale, v_wind_scale):
    nu_syn = 240.0 * ((epsilon_B/epsilon_B_scale)**(-3.0/2.0)) * ((Mdot/Mdot_scale)**(-3.0/2.0)) * ((v_wind/v_wind_scale)**(3.0/2.0))
    return nu_syn

def t_syn_ratio(epsilon_B, Mdot, v_wind, nu, epsilon_B_scale, Mdot_scale, v_wind_scale, nu_scale):
    t_syn_ratio = 2.0 * ((epsilon_B/epsilon_B_scale)**(-3.0/4.0)) * ((Mdot/Mdot_scale)**(-3.0/4.0)) * ((v_wind/v_wind_scale)**(3.0/4.0)) * ((nu/nu_scale)**(-0.5))
    return t_syn_ratio

Inverse Compton cooling dominant:

In [41]:
def nu_IC(epsilon_B, Mdot, v_wind, v_ej, L_bol, epsilon_B_scale, Mdot_scale, v_wind_scale, v_ej_scale, L_bol_scale):
    nu_IC = 8.0 * ((epsilon_B/epsilon_B_scale)**(0.5)) * ((Mdot/Mdot_scale)**(0.5)) * ((v_wind/v_wind_scale)**(-0.5)) * ((v_ej/v_ej_scale)**(4.0)) * (L_bol/L_bol_scale)
    return nu_IC

def t_IC_ratio(L_bol, epsilon_B, Mdot, v_wind, v_ej, nu, L_bol_scale, epsilon_B_scale, Mdot_scale, v_wind_scale, v_ej_scale, nu_scale):
    t_IC_ratio = 0.18 * ((L_bol/L_bol_scale)**(-1.0)) * ((epsilon_B/epsilon_B_scale)**(1.0/4.0)) * ((Mdot/Mdot_scale)**(1.0/4.0)) * ((v_wind/v_wind_scale)**(-1.0/4.0)) * ((v_ej/v_ej_scale)**(2.0)) * ((nu/nu_scale)**(-0.5))
    return t_IC_ratio

## Load in physical parameter values to use in calculations:

Load in model-independent parameters:

In [48]:
# physical values:
D = 10.1 # in Mpc, as adopted by Maeda et al. 2023 and Boestrom et al. 2020
D_err = 1.8 # ^^
v_wind = 20 # km/s, use 20 b/c it is used in Maeda+2023a
v_ej = 7000 # km/s, adopted from Boestrom et al. 2020
T_e = 10**5 # K, standard value to adopt
epsilon_B = 0.012 # from Maeda+2023a, Model A
# m_H = # ???
# L_bol = ??

# scale factors:
F_p_scale = 1 # Jy
D_scale = 1 # Mpc
nu_p_scale = 10 # GHz, consistent with scale used for fitting
v_wind_scale = 10 # km/s, should this be the same or different from the actual value adopted?
v_ej_scale = 10**4 # km/s, check with Poonam??
T_e_scale = 10**5 # K, check with Poonam??
t_scale = 100 # days, check with Poonam??
epsilon_B_scale = 0.1
B_scale = 1 # Gauss?
# Mdot_scale = ??
# L_bol_scale = ??

Load in the parameter info for SSA dominant absorption:

In [43]:
# both epochs:
p = 3.30

# ~1360 days:
nu_p_SSA_early = 11.91
F_p_SSA_early = 7.46

# ~1700 days:
nu_p_SSA_late = 11.91
F_p_SSA_late = 6.05

Load in the parameter info for FFA dominant absorption:

In [44]:
# define a function that just returns tau:
def tau_FFA(freq, K2):
    #scale the freqs by 10
    tau = K2 * (freq/10.0)**(-2.1)
    return tau

# both epochs:
alpha = 1.20

# ~1360 days:
K1_FFA_early = 13.71
K2_FFA_early = 0.65
tau_early_10GHz = tau_FFA(10, K2_FFA_early) # choose tau at 10 GHz, our scaling factor

# ~1700 days:
K1_FFA_late = 11.30
K2_FFA_late = 0.66
tau_late_10GHz = tau_FFA(10, K2_FFA_late) # choose tau at 10 GHz, our scaling factor

## Calculations:

Calculate B, R, Mdot for SSA:

In [47]:
# ~1360 days:
t_early = 1360
B_SSA_early = B_peak_SSA(p, F_p_SSA_early, D, nu_p_SSA_early, F_p_scale, D_scale, nu_p_scale)
R_SSA_early = R_peak_SSA(p, F_p_SSA_early, D, nu_p_SSA_early, F_p_scale, D_scale, nu_p_scale)
# Mdot_SSA_early = Mdot_peak_SSA(m_H, B_SSA_early, t_early, v_wind, B_scale, t_scale, v_wind_scale)

physical_params_SSA_early = {
    'B':(B_SSA_early, 'Gauss'),
    'R':(R_SSA_early, 'cm'),
    'Mdot':(0, 'solar masses per year')
}

# ~1700 days:
t_late = 1700
B_SSA_late = B_peak_SSA(p, F_p_SSA_late, D, nu_p_SSA_late, F_p_scale, D_scale, nu_p_scale)
R_SSA_late = R_peak_SSA(p, F_p_SSA_late, D, nu_p_SSA_late, F_p_scale, D_scale, nu_p_scale)
# Mdot_SSA_early = Mdot_peak_SSA(m_H, B_SSA_late, t_late, v_wind, B_scale, t_scale, v_wind_scale)

physical_params_SSA_late = {
    'B':(B_SSA_late, 'Gauss'),
    'R':(R_SSA_late, 'cm'),
    'Mdot':(0, 'solar masses per year')
}

physical_params_SSA_late

{'B': (0.5860349453673143, 'Gauss'),
 'R': (1.5373505581688842e+17, 'cm'),
 'Mdot': (0, 'solar masses per year')}

Calculate Mdot for FFA:

In [49]:
# ~1360 days:
t_early = 1360
Mdot_FFA_early = Mdot_peak_FFA(tau_early_10GHz, v_ej, t_early, T_e, v_wind, v_ej_scale, t_scale, T_e_scale, v_wind_scale)
Mdot_FFA_early

0.002254493820637841